## Imports

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np
import gzip

## Data

In [32]:
def open_images(filename):
    with gzip.open(filename, "rb") as file:
        data = file.read()
        return np.frombuffer(data, dtype=np.uint8, offset=16)\
            .reshape(-1, 28, 28)\
            .astype(np.float32)


def open_labels(filename):
    with gzip.open(filename, "rb") as file:
        data = file.read()
        return np.frombuffer(data, dtype=np.uint8, offset=8)
    
X_train = open_images("../data/fashion/train-images-idx3-ubyte.gz")
y_train = open_labels("../data/fashion/train-labels-idx1-ubyte.gz") 
y_train = to_categorical(y_train)

 
X_test = open_images("../data/fashion/t10k-images-idx3-ubyte.gz")
y_test = open_labels("../data/fashion/t10k-labels-idx1-ubyte.gz")
y_test = to_categorical(y_test)

## Model

In [33]:
model = Sequential()
model.add(Conv2D(20, kernel_size = (5, 5), activation = "relu",  input_shape = (28, 28, 1)))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(10, kernel_size = (5, 5), activation = "relu"))
model.add(Flatten())
model.add(Dense(10, activation = "softmax"))

model.summary()
model.compile(optimizer = "rmsprop", loss = "categorical_crossentropy", metrics=["accuracy"])   


Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_45 (Conv2D)           (None, 24, 24, 20)        520       
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 12, 12, 20)        0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 8, 8, 10)          5010      
_________________________________________________________________
flatten_23 (Flatten)         (None, 640)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 10)                6410      
Total params: 11,940
Trainable params: 11,940
Non-trainable params: 0
_________________________________________________________________


## Train and Test

In [26]:
model.fit(
         X_train.reshape(60000,28,28,1),
         y_train,
         epochs=50,
         batch_size = 1000) 

print("\nTraining Set Accuracy: {}".format(model.evaluate(X_train.reshape(-1,28,28,1),y_train)[1]))
print("\nTesting Set Accuracy: {}".format(model.evaluate(X_test.reshape(-1,28,28,1),y_test)[1]))

print("\nConfusion Matrix:")
predictions = model.predict(X_test.reshape(-1,28,28,1))
ytrue = pd.Series(np.argmax(y_test, axis=1), name="actual")
ypred = pd.Series(np.argmax(predictions, axis=1), name="predicted")
pd.crosstab(ytrue,ypred)

Epoch 1/50
60/60 [==============================] - 6s 97ms/step - loss: 2.0716 - accuracy: 0.5920
Epoch 2/50
60/60 [==============================] - 6s 102ms/step - loss: 0.7027 - accuracy: 0.7652
Epoch 3/50
60/60 [==============================] - 6s 98ms/step - loss: 0.5428 - accuracy: 0.8093
Epoch 4/50
60/60 [==============================] - 6s 97ms/step - loss: 0.4586 - accuracy: 0.8364
Epoch 5/50
60/60 [==============================] - 6s 99ms/step - loss: 0.4075 - accuracy: 0.8528
Epoch 6/50
60/60 [==============================] - 6s 98ms/step - loss: 0.3733 - accuracy: 0.8647
Epoch 7/50
60/60 [==============================] - 6s 99ms/step - loss: 0.3479 - accuracy: 0.8733
Epoch 8/50
60/60 [==============================] - 6s 106ms/step - loss: 0.3264 - accuracy: 0.8801
Epoch 9/50
60/60 [==============================] - 7s 109ms/step - loss: 0.3132 - accuracy: 0.8850
Epoch 10/50
60/60 [==============================] - 7s 112ms/step - loss: 0.3012 - accuracy: 0.8902
Epoch

predicted,0,1,2,3,4,5,6,7,8,9
actual,,,,,,,,,,
0,846,1,19,26,6,1,91,0,10,0
1,2,975,0,16,3,0,2,0,2,0
2,23,1,804,13,90,0,67,0,2,0
3,20,7,14,916,16,0,22,0,5,0
4,1,0,53,45,827,0,71,0,3,0
5,1,0,0,0,0,964,0,13,3,19
6,145,3,66,36,84,0,649,0,17,0
7,0,0,0,0,0,27,0,928,2,43
8,6,4,5,4,6,3,14,1,956,1
